In [1]:
from pprint import pprint
from transformers import AutoModelForObjectDetection, AutoFeatureExtractor
import torchvision
from pathlib import Path
import torch

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="onQHDhSGeUEnGY4e1Qq7")
project = rf.workspace("leafframes").project("leaf-area-zxdye")
version = project.version(3)
dataset = version.download("coco")
                

loading Roboflow workspace...
loading Roboflow project...


In [3]:
MODEL_NAME = 'facebook/detr-resnet-50'

model = AutoModelForObjectDetection.from_pretrained(MODEL_NAME)
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
The class DetrFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use DetrImageProcessor instead.


In [7]:
# feature_extractor, 
model

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [8]:
class LeafDetectionCocoDataset(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor):
        ann_file = img_folder / '_annotations.coco.json'
        super(LeafDetectionCocoDataset, self).__init__(root=img_folder, annFile=ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        img, targets = super(LeafDetectionCocoDataset, self).__getitem__(idx)
        image_id = self.ids[idx]
        annotations = {'image_id': image_id, 'annotations': targets}
        encoding = self.feature_extractor(images=img, annotations=annotations, return_tensor='pt')
        pixel_values = encoding['pixel_values'][0]
        target = encoding['labels'][0]
        return pixel_values, target

In [31]:
dataset = LeafDetectionCocoDataset(img_folder=Path('/home/piotr/projects/leaf-area/leaf-area/notebooks/leaf-area-1/train'),
                                   feature_extractor=feature_extractor)

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [32]:
pixel_values, target = dataset[0]
print(pixel_values.shape)
pprint(target)

Unused or unrecognized kwargs: return_tensor.


(3, 800, 1066)
{'area': array([3118.05  , 4606.8257, 5464.3164, 4238.6294, 4709.7305, 2649.2234,
       3311.4226, 4292.427 , 4631.9834, 5664.298 , 4451.19  , 5432.3364,
       3254.285 ], dtype=float32),
 'boxes': array([[0.441625  , 0.23850003, 0.02924998, 0.12500001],
       [0.666     , 0.26833335, 0.03700002, 0.146     ],
       [0.31137496, 0.28466666, 0.04125   , 0.15533333],
       [0.560375  , 0.2911667 , 0.03525003, 0.141     ],
       [0.42125   , 0.4506667 , 0.038     , 0.14533333],
       [0.74462503, 0.45533335, 0.02725002, 0.11400001],
       [0.50425   , 0.47783333, 0.03300004, 0.11766666],
       [0.585125  , 0.4873334 , 0.03775002, 0.13333333],
       [0.3435    , 0.5378334 , 0.03550002, 0.15300003],
       [0.707     , 0.7206667 , 0.04100005, 0.162     ],
       [0.453125  , 0.7386667 , 0.03575   , 0.14599998],
       [0.58100003, 0.74700004, 0.03899997, 0.16333328],
       [0.29924998, 0.7626667 , 0.03600001, 0.10600006]], dtype=float32),
 'class_labels': array([1, 

In [33]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    pixel_values = [item[0] for item in batch]
    encoding = feature_extractor.pad(pixel_values, return_tensors='pt')
    labels = [item[1] for item in batch]
    return {
        'pixel_values': encoding['pixel_values'],
        'labels': labels
    }

In [34]:
dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size = 2)
batch = next(iter(dataloader))

Unused or unrecognized kwargs: return_tensor.
Unused or unrecognized kwargs: return_tensor.


In [35]:
print(batch['pixel_values'].shape)
pprint(batch['labels'])

torch.Size([2, 3, 800, 1066])
[{'area': array([3118.05  , 4606.8257, 5464.3164, 4238.6294, 4709.7305, 2649.2234,
       3311.4226, 4292.427 , 4631.9834, 5664.298 , 4451.19  , 5432.3364,
       3254.285 ], dtype=float32),
  'boxes': array([[0.441625  , 0.23850003, 0.02924998, 0.12500001],
       [0.666     , 0.26833335, 0.03700002, 0.146     ],
       [0.31137496, 0.28466666, 0.04125   , 0.15533333],
       [0.560375  , 0.2911667 , 0.03525003, 0.141     ],
       [0.42125   , 0.4506667 , 0.038     , 0.14533333],
       [0.74462503, 0.45533335, 0.02725002, 0.11400001],
       [0.50425   , 0.47783333, 0.03300004, 0.11766666],
       [0.585125  , 0.4873334 , 0.03775002, 0.13333333],
       [0.3435    , 0.5378334 , 0.03550002, 0.15300003],
       [0.707     , 0.7206667 , 0.04100005, 0.162     ],
       [0.453125  , 0.7386667 , 0.03575   , 0.14599998],
       [0.58100003, 0.74700004, 0.03899997, 0.16333328],
       [0.29924998, 0.7626667 , 0.03600001, 0.10600006]], dtype=float32),
  'class_l

In [36]:
import pytorch_lightning as pl
import numpy as np

In [37]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS

class LeafAreaDetrDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)

    def collate_fn(self, batch):
        pixel_values = [item[0] for item in batch]
        encoding = self.feature_extractor.pad(pixel_values, return_tensors='pt')
        labels = [item[1] for item in batch]
        batch = {}
        batch['pixel_values'] = encoding['pixel_values']
        batch['labels'] = labels
        return batch
    
    def train_dataloader(self):
        train_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/train'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(train_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=True)

    def val_dataloader(self):
        valid_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/valid'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(valid_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=False)

    def test_dataloader(self):
        test_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/test'),
                                                 feature_extractor=self.feature_extractor)
        return DataLoader(test_dataset, collate_fn=self.collate_fn, batch_size=2, shuffle=False)
    
    @property
    def num_labels(self):
        train_dataset = LeafDetectionCocoDataset(img_folder=Path('leaf-area-3/train'),
                                                 feature_extractor=self.feature_extractor)
        # return np.unique(list(train_dataset.coco.cats.keys()))
        return train_dataset.coco.cats.keys()
        
        

In [38]:
datamodule = LeafAreaDetrDataModule()
datamodule.num_labels

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


dict_keys([0, 1])

In [39]:
class Detr(pl.LightningModule):
    def __init__(self, 
                 num_labels,
                 lr, weight_decay):
        super().__init__()
        self.model = AutoModelForObjectDetection.from_pretrained(
            MODEL_NAME,
            num_labels=num_labels,
            ignore_mismached_sizes=True
        )
        self.lr = lr
        self.weight_decay = weight_decay

    def forward(self, pixel_values):
        outputs = self.model(pixel_values=pixel_values)
        return outputs
    
    def common_step(self, batch):
        pixel_values = batch['pixel_values']
        labels = [{k: v.to(self.device) for k, v in target.items()} for target in batch['labels']]
        outputs = self.model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss
        loss_dict = outputs.loss_dict
        return loss, loss_dict
    
    def training_step(self, batch):
        loss, loss_dict = self.common_step(batch)
        self.log('training_loss', loss)
        for k, v in loss_dict.items():
            self.log('train_' + k, v.item())
        return loss
    
    def validation_step(self, batch):
        loss, loss_dict = self.common_step(batch)
        self.log('validation_loss', loss)
        for k, v in loss_dict.items():
            self.log('validation_' + k, v.item())
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.paramters(),
                                      lr=self.lr,
                                      weight_decay=self.weight_decay)
        return optimizer

In [40]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, Callback

model = Detr(num_labels=datamodule.num_labels,
             lr=2.5e-5,
             weight_decay=1e-4)

trainer = Trainer(max_steps=20000,
                  gradient_clip_val=0.1,
                  accumulate_grad_batches=4,
                  callbacks=[
                      EarlyStopping(monitor='validation_loss', patience=3, mode='min'),
                      ModelCheckpoint(monitor='validation_loss', mode='min', save_top_k=1,dirpath='checkpoints'),
                  ])

trainer.fit(model, datamodule)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


TypeError: cannot pickle 'dict_keys' object